Lien de téléchargement des données: https://cvml.ista.ac.at/AwA2/        

13GB file : https://cvml.ista.ac.at/AwA2/AwA2-data.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.0 MB/s eta 0:00:00


In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
import sys
print(sys.version)

3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]


In [ ]:
import sys
import cv2 # Pour utiliser opencv-python, il faut la version de python est 3.7
import os
import csv

import numpy as np 
import pandas as pd 
import math

import torch 
from torch.utils.data import Dataset, DataLoader
import torchvision 
from torchvision.io import read_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim

np.random.seed(0)

In [ ]:
# Constant. Should be the path to the folder named JPEGImages, containing the 33K images in its subfolders.
DATA_FOLDER_PATH = '/content/drive/MyDrive/IFT3710/Animals_with_Attributes2/'
JPEGIMAGES_FOLDER_PATH = '/content/drive/MyDrive/IFT3710/Animals_with_Attributes2/JPEGImages/'

In [ ]:
# quick test
test = JPEGIMAGES_FOLDER_PATH+"fox/fox_10001.jpg"
img = cv2.imread(test) 
print(img.shape) #ndarray
print(type(img))
# cv2.imshow('Sample Image from AwA2 dataset',img)
# cv2.waitKey(0)

(764, 918, 3)
<class 'numpy.ndarray'>


In [ ]:
labels_dirs = os.listdir(JPEGIMAGES_FOLDER_PATH)
print(labels_dirs)
len(labels_dirs) # 50 labels / subdirectories

['elephant', 'lion', 'deer', 'cow', 'squirrel', 'german+shepherd', 'skunk', 'horse', 'mole', 'walrus', 'weasel', 'grizzly+bear', 'rabbit', 'buffalo', 'beaver', 'chimpanzee', 'bobcat', 'wolf', 'bat', 'mouse', 'seal', 'collie', 'spider+monkey', 'otter', 'rat', 'tiger', 'blue+whale', 'hamster', 'ox', 'zebra', 'sheep', 'giraffe', 'chihuahua', 'leopard', 'polar+bear', 'dolphin', 'fox', 'siamese+cat', 'persian+cat', 'pig', 'moose', 'killer+whale', 'giant+panda', 'humpback+whale', 'antelope', 'raccoon', 'dalmatian', 'rhinoceros', 'gorilla', 'hippopotamus']


50

# Note : Some labels have a low number of images. 

## Possible solutions to explore : 
    Data augmentation : creating new training data by applying random transformations to existing images, such as rotating, cropping, or flipping them.

In [ ]:
def find_num_images_per_label(img_dir = JPEGIMAGES_FOLDER_PATH): #-> tuple[dict,dict]: 
    """ 
    USEFUL FOR SAMPLING.
    Return a dict with keys as the 50 labels, and values being the number of images in each subdirectory corresponding to label
    and a second dict with the relative numbers (proportion) for every label compared to the total number of images (useful for sampling)"""
    labels_dirs = os.listdir(img_dir)
    num_images_per_label = dict.fromkeys(labels_dirs)
    proportions_images_per_label = dict.fromkeys(labels_dirs)
    total_num_images = 0

    # Update absolute number of images per label
    for i, label in enumerate(labels_dirs) : 
        specific_label_path = os.path.join(img_dir, labels_dirs[i])
        num_images_label = len(os.listdir(specific_label_path))
        total_num_images += num_images_label
        num_images_per_label[label] = num_images_label

    # Update relative number of images per label (proportion)
    for i, label in enumerate(labels_dirs) : 
        num_images_label = num_images_per_label[label]
        proportion_label = round(num_images_label / total_num_images, 4)
        proportions_images_per_label[label] = proportion_label

    return num_images_per_label, proportions_images_per_label

num_images_per_label, proportions_images_per_label = find_num_images_per_label()
print(num_images_per_label)
print(proportions_images_per_label)

{'elephant': 1038, 'lion': 1019, 'deer': 1344, 'cow': 1338, 'squirrel': 1200, 'german+shepherd': 1033, 'skunk': 188, 'horse': 1645, 'mole': 100, 'walrus': 215, 'weasel': 272, 'grizzly+bear': 852, 'rabbit': 1088, 'buffalo': 895, 'beaver': 193, 'chimpanzee': 728, 'bobcat': 630, 'wolf': 589, 'bat': 383, 'mouse': 185, 'seal': 988, 'collie': 1028, 'spider+monkey': 291, 'otter': 758, 'rat': 310, 'tiger': 877, 'blue+whale': 174, 'hamster': 779, 'ox': 728, 'zebra': 1170, 'sheep': 1420, 'giraffe': 1202, 'chihuahua': 567, 'leopard': 720, 'polar+bear': 868, 'dolphin': 946, 'fox': 664, 'siamese+cat': 500, 'persian+cat': 747, 'pig': 713, 'moose': 704, 'killer+whale': 291, 'giant+panda': 874, 'humpback+whale': 709, 'antelope': 1046, 'raccoon': 512, 'dalmatian': 549, 'rhinoceros': 696, 'gorilla': 872, 'hippopotamus': 684}
{'elephant': 0.0278, 'lion': 0.0273, 'deer': 0.036, 'cow': 0.0359, 'squirrel': 0.0322, 'german+shepherd': 0.0277, 'skunk': 0.005, 'horse': 0.0441, 'mole': 0.0027, 'walrus': 0.0058, 

In [ ]:
ANNOTATIONS_FILENAME = 'annotations.csv'

def create_annotations_csv_file(annotations_filename = ANNOTATIONS_FILENAME, img_dir = JPEGIMAGES_FOLDER_PATH): 
    """ 
    Create a csv annotations_file, annotations.csv, with two columns, in the format : 
                        path/to/image, label
    
    The annotation csv is necessary for DataLoader.
    """
    
    labels_dirs:list = os.listdir(img_dir)
   
    if os.path.exists(annotations_filename):
        os.remove(annotations_filename)
        print(f'Deleted existent {ANNOTATIONS_FILENAME} file.\n ---------------------------')
    
    with open(annotations_filename, 'w', newline='') as file :
        writer = csv.writer(file, dialect='excel', delimiter=',')

        for i, label in enumerate(labels_dirs) : 

            specific_label_path = os.path.join(img_dir, label)
            images_names = os.listdir(specific_label_path)

            for j, image_name in enumerate(images_names):
                full_path_to_img= os.path.join(specific_label_path, image_name)
                full_path_to_img= os.path.join(label, image_name)

                row = [full_path_to_img, label]
                writer.writerow(row)

    print(f'Sucessfully created {ANNOTATIONS_FILENAME} file.')

#
create_annotations_csv_file()

Deleted existent annotations.csv file.
 ---------------------------
Sucessfully created annotations.csv file.


In [ ]:
# labels_in_number = pd.read_csv(DATA_FOLDER_PATH+"classes.txt", delim_whitespace=True,header=None)
labels_dict = {}
with open(DATA_FOLDER_PATH+"classes.txt") as f:
    for line in f:
        # print(line.split())
        (key,val) = line.split()
        labels_dict[val] = int(key)-1
print(labels_dict)

{'antelope': 0, 'grizzly+bear': 1, 'killer+whale': 2, 'beaver': 3, 'dalmatian': 4, 'persian+cat': 5, 'horse': 6, 'german+shepherd': 7, 'blue+whale': 8, 'siamese+cat': 9, 'skunk': 10, 'mole': 11, 'tiger': 12, 'hippopotamus': 13, 'leopard': 14, 'moose': 15, 'spider+monkey': 16, 'humpback+whale': 17, 'elephant': 18, 'gorilla': 19, 'ox': 20, 'fox': 21, 'sheep': 22, 'seal': 23, 'chimpanzee': 24, 'hamster': 25, 'squirrel': 26, 'rhinoceros': 27, 'rabbit': 28, 'bat': 29, 'giraffe': 30, 'wolf': 31, 'chihuahua': 32, 'rat': 33, 'weasel': 34, 'otter': 35, 'buffalo': 36, 'zebra': 37, 'giant+panda': 38, 'deer': 39, 'bobcat': 40, 'pig': 41, 'lion': 42, 'mouse': 43, 'polar+bear': 44, 'collie': 45, 'walrus': 46, 'raccoon': 47, 'cow': 48, 'dolphin': 49}


In [ ]:
class AWA2Dataset(Dataset): # Dataset class to serve as input for the DataLoader.
    """ 
    Dataset class to serve as input for the DataLoader.
    Implements all the required methods and more. 
    """

    def __init__(self, annotations_file=ANNOTATIONS_FILENAME, img_dir=JPEGIMAGES_FOLDER_PATH, 
                transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

        numbers_infos_dicts: tuple[dict,dict] = find_num_images_per_label(img_dir=JPEGIMAGES_FOLDER_PATH)
        self.num_images_per_label = numbers_infos_dicts[0]
        self.proportions_images_per_label = numbers_infos_dicts[1]

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        # img_path = self.img_labels.iloc[idx, 0]
        key = self.img_labels.iloc[idx, 1]

        # Mapping the labels from string to tensor
        label = labels_dict[key]

        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
dataset = AWA2Dataset()
image,label = dataset[4125]



## TODO : Change transforms. Currently this is not useful.
dataset.transform = transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.Resize((224, 224)),
                        transforms.RandomHorizontalFlip(),
                        transforms.Grayscale(num_output_channels=3),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])

# Testing. All good
random_index = np.random.randint(0, len(dataset))
image, label = dataset[random_index]

In [ ]:
# Experiment with DataLoader. Everything works good
dataloader = DataLoader(dataset = dataset, batch_size=4, shuffle=True)
dataiter = iter(dataloader)
data = next(dataiter)

images, labels = data 

### ViT ###

In [ ]:
pip install vit-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 4.9 MB/s eta 0:00:00


In [ ]:
import torch.nn as nn
# import timm
from vit_pytorch import ViT
from vit_pytorch import SimpleViT

vit_model = SimpleViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048
)

vit_model.eval

<bound method Module.eval of SimpleViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b h w (p1 p2 c)', p1=32, p2=32)
    (1): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=3072, out_features=1024, bias=True)
    (3): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (transformer): Transformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): Attention(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attend): Softmax(dim=-1)
          (to_qkv): Linear(in_features=1024, out_features=3072, bias=False)
          (to_out): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (1): FeedForward(
          (net): Sequential(
            (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=1024, out_features=2048, bias=True)
            (2): GELU(approximate='none')
            (3): Linear(in_features=2048

### CUDA ###

In [ ]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [ ]:
torch.cuda.is_available()

True

### Question: nombre de noeud ###

In [ ]:
nb_noeud = 4096

In [ ]:
# vit_model.classifier[4] = torch.nn.Linear(4096,nb_noeud)
# vit_model.classifier[6] = torch.nn.Linear(nb_noeud,50)

In [ ]:
import sys
print(sys.version)

3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]


In [ ]:
vit_model.to(device)

SimpleViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b h w (p1 p2 c)', p1=32, p2=32)
    (1): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=3072, out_features=1024, bias=True)
    (3): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (transformer): Transformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): Attention(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attend): Softmax(dim=-1)
          (to_qkv): Linear(in_features=1024, out_features=3072, bias=False)
          (to_out): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (1): FeedForward(
          (net): Sequential(
            (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=1024, out_features=2048, bias=True)
            (2): GELU(approximate='none')
            (3): Linear(in_features=2048, out_features=1024, bias=Tru

### Question: check dimension de dataloader   ###

###  Split training data and test data ###

In [ ]:
batch_size = 32

In [ ]:
dataloader = DataLoader(dataset = dataset, batch_size=batch_size, shuffle=True)
train_size = int(0.8*len(dataset))
test_size = len(dataset) - train_size

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset,[train_size,test_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, num_workers=0)

###   transfomer labels  ###

In [ ]:
path_class = DATA_FOLDER_PATH +"classes.txt"
class_animal = pd.read_table(path_class,header= None)

In [ ]:
animals = class_animal[1]

In [ ]:
dict_label_animal = {}

In [ ]:
n = 0
for i in range(0,len(animals)):
    dict_label_animal[animals[i]] = n
    n+=1

In [ ]:
def label_to_num(tuple_labels):
    list_labels =[]
    for tuple_label in tuple_labels:
        list_labels.append(dict_label_animal[tuple_label])
    return torch.tensor(list_labels) 

###   Loss function  ###

In [ ]:
lr = 0.001

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(vit_model.parameters(), lr= lr , momentum=0.9)

###  Training  ###

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="vit-basic",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.001,
    "architecture": "ViT",
    "dataset": "AWA2",
    "epochs": 50,
    }
)

for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data[0].to(device)
        labels = data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = vit_model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss ))
        
        # log metrics to wandb
        wandb.log({"loss": running_loss})

        running_loss = 0.0


# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

print('Finished Training of ViT')

loss,▄▃▅▄▆▂▆▅▆▆█▄▆▇▆▄▃▂▆▅▆▅▄▂▅▄▅▄▆▅▅▇▄▃▄▁▄▇▅▂
loss,3.70026


[1,     1] loss: 3.724
[1,     2] loss: 3.737
[1,     3] loss: 3.645
[1,     4] loss: 3.563
[1,     5] loss: 3.824
[1,     6] loss: 3.474
[1,     7] loss: 3.654
[1,     8] loss: 4.012
[1,     9] loss: 3.612
[1,    10] loss: 3.872
[1,    11] loss: 3.768
[1,    12] loss: 3.700
[1,    13] loss: 3.895
[1,    14] loss: 3.719
[1,    15] loss: 3.997
[1,    16] loss: 3.689
[1,    17] loss: 3.747
[1,    18] loss: 3.515
[1,    19] loss: 3.859
[1,    20] loss: 3.713
[1,    21] loss: 4.036
[1,    22] loss: 3.588
[1,    23] loss: 3.767
[1,    24] loss: 3.893
[1,    25] loss: 3.594
[1,    26] loss: 3.686
[1,    27] loss: 3.820
[1,    28] loss: 3.756
[1,    29] loss: 3.598
[1,    30] loss: 3.692
[1,    31] loss: 3.618
[1,    32] loss: 3.940
[1,    33] loss: 3.870
[1,    34] loss: 3.603
[1,    35] loss: 3.781
[1,    36] loss: 3.799
[1,    37] loss: 3.710
[1,    38] loss: 3.672
[1,    39] loss: 3.860
[1,    40] loss: 3.849
[1,    41] loss: 3.794
[1,    42] loss: 3.654
[1,    43] loss: 3.711
[1,    44] 

### Test ###

In [ ]:
correct = 0
total = 0
wrong_pred =[]
right_label = []
with torch.no_grad():
    for data in test_loader:
        images = data[0].to(device)
        labels = data[1].to(device)
        outputs = vit_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        
        for i in range(0,len(predicted)):
              if predicted[i].item() != labels[i].item():
                    wrong_pred.append(predicted[i].item())
                    right_label.append(labels[i].item())
        

        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

###   Analyse  ###

In [ ]:
nb_wrong_pred = []
for i in range(0,50):
    nb_wrong_pred.append(wrong_pred.count(i))

In [ ]:
list_animal = list(dict_label_animal.keys())

In [ ]:
plt.bar(range(50), nb_wrong_pred)
plt.show()

In [ ]:
good_classification = []
bad_classification = []
for i in range(50):
    if nb_wrong_pred[i]<=5:
        good_classification.append(i)
    if    nb_wrong_pred[i]>=60:
        bad_classification.append(i)

In [ ]:
def find_right_animal(m):
    wrong_pred_m =[]
    for j in [i for i,x in enumerate(wrong_pred) if x == m]:
        wrong_pred_m.append(right_label[j])
    return list_animal[max(wrong_pred_m,key = wrong_pred_m.count)]

In [ ]:
for i in good_classification :
    print('ViT a bien classifie '+animals[i])

In [ ]:
for i in bad_classification:
    print('ViT a mal classifie '+animals[i]+' , melange souvent avec '+find_right_animal(i))